# Бэггинг

Пусть есть случайные одинаково распределённые величины $\xi_1, \xi_2, \dots, \xi_n$, скоррелированные с коэффициентом корреляции $\rho$ и дисперсией $\sigma^2$. Какова будет дисперсия величины $\frac1n \sum_{i=1}^n \xi_i$?

$$\mathbf{D} \frac1n \sum_{i=1}^n \xi_i = \frac1{n^2}\mathbf{cov} (\sum_{i=1}^n \xi_i, \sum_{i=1}^n \xi_i) = \frac1{n^2} \sum_{i=1, j=1}^n \mathbf{cov}(\xi_i, \xi_j) = \frac1{n^2} \sum_{i=1}^n \mathbf{cov}(\xi_i, \xi_i) + \frac1{n^2} \sum_{i=1, j=1, i\neq j}^n \mathbf{cov}(\xi_i, \xi_j) = \frac1{n^2} \sum_{i=1}^n \sigma^2+ \frac1{n^2} \sum_{i=1, j=1, i\neq j}^n \rho \sigma^2 =$$
$$ = \frac1{n^2} n \sigma^2 + \frac1{n^2} n(n-1) \rho \sigma^2  = \frac{\sigma^2( 1 + \rho(n-1))}{n}$$

Таким образом, чем менее величины скоррелированы между собой, тем меньше будет дисперсия после их усреднения. Грубо говоря в этом и состоит идея бэггинга: давайте сделаем много максимально независимых моделей, а потом их усредим, и тогда предсказания станет более устойчивым!

# Бэггинг над решающими деревьями

Посмотрим, какие модели можно получить из деревьев с помощью их рандомизации

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('HR.csv')

target = 'left'
features = [c for c in data if c != target]
print(features)

X, y = data[features], data[target]

['last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']


In [3]:
rnd_d3 = DecisionTreeClassifier(max_features=int(len(features) ** 0.5)) # Решающее дерево с рандомизацией в сплитах
d3 = DecisionTreeClassifier() # Обычное решающее дерево

Качество классификации решающим деревом с настройками по-умолчанию:

In [4]:
print("Decision tree:", cross_val_score(d3, X, y).mean())

Decision tree: 0.6523099419883976


Бэггинг над решающими деревьями:

In [5]:
print("D3 bagging:", cross_val_score(BaggingClassifier(d3, random_state=42), X, y).mean())

D3 bagging: 0.7174495299059812


Усредненная модель оказалась намного лучше. Оказывается, у решающих деревьев есть существенный недостаток - нестабильность получаемого дерева при небольших изменениях в выборке. Но бэггинг обращает этот недостаток в достоинство, ведь усредненная модель работает лучше, когда базовые модели слабо скоррелированы.

Изучив параметры DecisionTreeClassifier, можно найти хороший способ сделать деревья еще более различными - при построении каждого узла отбирать случайные max_features признаков и искать информативное разбиение только по одному из них.

In [6]:
print("Randomized D3 Bagging:", cross_val_score(BaggingClassifier(rnd_d3, random_state=42), X, y).mean())

Randomized D3 Bagging: 0.7194494632259785


В среднем, качество получается еще лучше. Для выбора числа признаков использовалась часто применяемая на практике эвристика - брать корень из общего числа признаков. Если бы мы решали задачу регрессии - брали бы треть от общего числа.

In [7]:
print("Random Forest:", cross_val_score(RandomForestClassifier(random_state=42), X, y).mean())

Random Forest: 0.7232495965859839


In [8]:
print("Logistic Regression:", cross_val_score(LogisticRegression(), X, y).mean())

Logistic Regression: 0.6287053143962126


## Опциональное задание
Повторные запуски cross_val_score будут показывать различное качество модели.

Это зависит от параметра рандомизации модели "random_state" в DecisionTreeClassifier, BaggingClassifie или RandomForest.

Чтобы понять, действительно ли одна модель лучше другой, можно посмотреть на её качество в среднем, то есть усредняя запуски с разным random_state. Предлагаю сравнить качество и понять, действительно ли BaggingClassifier(d3) лучше BaggingClassifier(rnd_d3)?

Также предлагаю ответить на вопрос, чем здесь отличается BaggingClassifier(rnd_d3) от RandomForestClassifier()?